<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf


    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance


    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-08-08 15:16:50
-------------------
qualified stocks: 92
with latest results: 28
still star stocks: 18
-------------------
Initial Investment:  1.20 C
CY Investment:  1.47 C
Reserve:  3.53 L
Current:  1.32 C
-------------------
Today PnL: -1.34 L (-1.01%)
Current PnL: -22.73 L (-15.46%)
CY Booked + Current PnL: -14.46 L (-9.83%)
-------------------
Total profit:  1.81 L
Total loss:  -24.54 L
-------------------
Total Booked + Current PnL: 12.52 L (10.43%)
Total Booked PnL: 35.25 L (29.36%)
Curr Year Booked PnL: 8.27 L (6.26%)
Prev Year Booked PnL: 26.98 L (22.48%)
Est FTT:  2.13 C
Est FTT PnL: 80.98 L (61.28%)
Deployed:  1.20 C
Current:  1.32 C
CAGR/XIRR %: 6.22%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,HINDALCO,651.95,756.01,-22.38,H-LC,5.62,107488.0,3176.0,13468.0,-1.47,3.04,12.53,15.96,10.0,0.24,0.84,20.46,X5K,ATH,METALS
67,SBIN,760.30,863.00,-12.46,M-LC,3.92,212967.0,11488.0,15738.0,-0.19,5.70,7.39,13.51,62.0,0.73,1.66,19.02,XY25,NTT,BANKS
71,SIS,477.00,477.00,2254.92,H-SC,7.09,63359.0,-17731.0,17728.0,0.32,-21.87,27.98,0.00,157.0,-1.00,0.49,28.41,OX40N,NTT,MISC
82,TTKPRESTIG,769.29,770.00,-9.13,H-SC,11.15,83067.0,-17710.0,17801.0,0.01,-17.57,21.43,0.09,145.0,-0.99,0.65,9.13,OX40N,NTT,DURABLES
55,MEDANTA,1087.93,1486.00,20.18,H-SC,13.41,153932.0,27732.0,18441.0,-0.86,21.97,11.98,36.59,156.0,1.50,1.20,37.24,XY24,NTT,HEALTHCARE


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,DABUR,505.20,735.0,-8.80,H-MC,4.51,197574.0,546.0,89066.0,-2.05,0.28,45.08,45.49,100.0,0.01,1.54,13.35,XY24,BTT,FMCG
91,WIPRO,243.46,420.0,-12.39,M-LC,6.25,150238.0,-707.0,110155.0,0.98,-0.47,73.32,72.51,55.0,-0.01,1.17,5.53,XR,NTT,IT


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,CIPLA,1492.70,1795.0,-22.84,H-LC,1.92,203267.0,-1233.0,42645.0,-0.49,-0.60,20.98,20.25,9.0,-0.03,1.58,8.08,X40N,BTT,PHARMA
5,ASIANPAINT,2961.56,4250.0,-15.33,H-LC,2.43,168286.0,-33100.0,120712.0,-1.26,-16.44,71.73,43.51,19.0,-0.27,1.31,17.14,X40,BTT,PAINTS
53,LTIM,5564.16,7230.2,-6.53,H-LC,3.55,180036.0,-20274.0,80260.0,-0.72,-10.12,44.58,29.94,24.0,-0.25,1.40,24.38,X200,ATH,IT
10,BAJAJHFL,122.26,152.0,-11.24,H-LC,3.66,184238.0,-16268.0,65036.0,-0.61,-8.11,35.30,24.33,30.0,-0.25,1.43,6.24,X40N,BTT,FINANCE
57,NESTLEIND,2268.60,2755.0,-59.66,H-LC,4.12,128092.0,-137334.0,194239.0,-2.01,-51.74,151.64,21.44,26.0,-0.71,1.00,3.59,XY25,NTT,FMCG


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,COFFEEDAY,59.14,80.0,-51.71,L-SC,21.99,70138.0,-43411.0,83464.0,3.08,-38.23,119.00,35.27,267.0,-0.52,0.55,71.66,XR,NTT,HOTELS
83,UJJIVANSFB,52.77,53.0,36.52,H-SC,20.68,118935.0,-23544.0,24168.0,1.87,-16.52,20.32,0.44,163.0,-0.97,0.92,40.38,OX40N,NTT,BANKS
8,AWL,325.39,485.0,-69.74,H-MC,5.25,209756.0,-58691.0,190375.0,1.42,-21.86,90.76,49.05,82.0,-0.31,1.63,7.85,XY24,NTT,FMCG
18,CAMPUS,294.86,393.0,-27.64,M-SC,2.68,150842.0,-16933.0,72781.0,1.09,-10.09,48.25,33.28,209.0,-0.23,1.17,16.78,XY24,NTT,FOOTWEAR
86,VAIBHAVGBL,350.21,521.0,43.25,H-SC,7.88,95641.0,-56350.0,130473.0,1.00,-37.07,136.42,48.77,127.0,-0.43,0.74,10.55,XR,NTT,JEWELLERY


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,LAOPALA,369.40,464.00,112.30,H-SC,3.52,67349.0,-33497.0,59321.0,-6.64,-33.22,88.08,25.61,141.0,-0.56,0.52,24.87,AR,NTT,CERAMICS
80,TITAGARH,1097.23,1548.00,-14.81,H-SC,6.24,139598.0,-52417.0,131306.0,-6.59,-27.30,94.06,41.08,158.0,-0.40,1.09,18.15,XY24,NTT,ENGINEERING
27,DIXON,14367.00,18940.15,-31.08,H-MC,6.51,126912.0,11976.0,24608.0,-4.80,10.42,19.39,31.83,85.0,0.49,0.99,38.55,X40N,ATH,IT
43,INDIGOPNTS,1407.73,1408.00,118.19,H-SC,3.97,143071.0,-31488.0,31519.0,-4.65,-18.04,22.03,0.02,135.0,-1.00,1.11,24.06,OX40N,NTT,PAINTS
76,TANLA,917.30,1963.11,-34.11,H-SC,7.90,161663.0,-77752.0,350712.0,-3.51,-32.48,216.94,114.01,133.0,-0.22,1.26,48.27,AR,ATH,IT


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,UJJIVANSFB,52.77,53.0,36.52,H-SC,20.68,118935.0,-23544.0,24168.0,1.87,-16.52,20.32,0.44,163.0,-0.97,0.92,40.38,OX40N,NTT,BANKS
82,TTKPRESTIG,769.29,770.0,-9.13,H-SC,11.15,83067.0,-17710.0,17801.0,0.01,-17.57,21.43,0.09,145.0,-0.99,0.65,9.13,OX40N,NTT,DURABLES
43,INDIGOPNTS,1407.73,1408.0,118.19,H-SC,3.97,143071.0,-31488.0,31519.0,-4.65,-18.04,22.03,0.02,135.0,-1.00,1.11,24.06,OX40N,NTT,PAINTS
77,TATAELXSI,7332.28,7332.0,-23.19,H-MC,2.17,80556.0,-22096.0,22088.0,-1.89,-21.53,27.42,-0.00,101.0,-1.00,0.63,22.45,OX40N,NTT,IT
71,SIS,477.00,477.0,2254.92,H-SC,7.09,63359.0,-17731.0,17728.0,0.32,-21.87,27.98,0.00,157.0,-1.00,0.49,28.41,OX40N,NTT,MISC


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,TCS,3794.03,4998.00,-27.38,H-LC,15.38,236722.0,-59212.0,153112.0,-0.40,-20.01,64.68,31.73,1.0,-0.39,1.84,1.06,X40,BTT,IT
88,VBL,492.64,671.64,-12.12,H-LC,11.03,266525.0,1485.0,94830.0,-1.45,0.56,35.58,36.33,3.0,0.02,2.07,13.93,X40N,ATH,FMCG
45,INFY,1461.46,2275.00,-20.67,H-LC,9.51,254645.0,-6956.0,152583.0,-1.00,-2.66,59.92,55.67,4.0,-0.05,1.98,3.28,X40,BTT,IT
1,ABB,5551.76,7934.00,-45.31,H-LC,9.48,195332.0,-21187.0,114093.0,-1.96,-9.79,58.41,42.91,5.0,-0.19,1.52,2.30,AR,NTT,ELECTRICAL
34,HCLTECH,1562.67,1943.91,1.41,H-LC,9.90,180011.0,-10635.0,57153.0,0.00,-5.58,31.75,24.40,8.0,-0.19,1.40,9.47,X40,ATH,IT


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.0,20.18,H-SC,13.41,153932.0,27732.0,18441.0,-0.86,21.97,11.98,36.59,156.0,1.5,1.2,37.24,XY24,NTT,HEALTHCARE


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,KPIGREEN,497.21,731.64,24.70,H-SC,13.18,127436.0,2139.0,56938.0,-1.38,1.71,44.68,47.15,144.0,0.04,0.99,59.43,MH,ATH,POWER
55,MEDANTA,1087.93,1486.00,20.18,H-SC,13.41,153932.0,27732.0,18441.0,-0.86,21.97,11.98,36.59,156.0,1.50,1.20,37.24,XY24,NTT,HEALTHCARE
35,HEROMOTOCO,4311.81,5949.07,0.00,H-MC,6.03,160892.0,9979.0,47318.0,-1.37,6.61,29.41,37.97,90.0,0.21,1.25,33.59,AR,ATH,AUTO
15,BLUESTARCO,1646.70,2326.38,-3.89,H-SC,7.95,180300.0,15630.0,52341.0,-1.34,9.49,29.03,41.28,119.0,0.30,1.40,18.23,X40N,ATH,AC
19,CAMS,3643.00,5211.76,-6.81,H-SC,2.41,104776.0,2772.0,41156.0,-1.73,2.72,39.28,43.06,120.0,0.07,0.81,20.30,X40N,ATH,MISC


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HEROMOTOCO,4311.81,5949.07,0.00,H-MC,6.03,160892.0,9979.0,47318.0,-1.37,6.61,29.41,37.97,90.0,0.21,1.25,33.59,AR,ATH,AUTO
42,INDIAMART,2327.09,4850.92,-47.89,H-SC,13.34,135643.0,12307.0,121455.0,-0.04,9.98,89.54,108.45,118.0,0.10,1.05,34.56,AR,ATH,MISC
50,KPIGREEN,497.21,731.64,24.70,H-SC,13.18,127436.0,2139.0,56938.0,-1.38,1.71,44.68,47.15,144.0,0.04,0.99,59.43,MH,ATH,POWER
90,WHIRLPOOL,1219.98,2270.00,-43.73,M-SC,4.90,94582.0,3084.0,75666.0,-2.60,3.37,80.00,86.07,211.0,0.04,0.74,36.82,XR,NTT,DURABLES
91,WIPRO,243.46,420.00,-12.39,M-LC,6.25,150238.0,-707.0,110155.0,0.98,-0.47,73.32,72.51,55.0,-0.01,1.17,5.53,XR,NTT,IT


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
71,SIS,477.00,477.00,2254.92,H-SC,7.09,63359.0,-17731.0,17728.0,0.32,-21.87,27.98,0.00,157.0,-1.00,0.49,28.41,OX40N,NTT,MISC
23,COFFEEDAY,59.14,80.00,-51.71,L-SC,21.99,70138.0,-43411.0,83464.0,3.08,-38.23,119.00,35.27,267.0,-0.52,0.55,71.66,XR,NTT,HOTELS
3,ALKYLAMINE,2347.98,4546.37,-4.84,H-SC,17.86,88567.0,-12396.0,106927.0,-2.31,-12.28,120.73,93.63,149.0,-0.12,0.69,31.92,SR,ATH,CHEMICALS
54,MASFIN,326.60,399.50,-20.05,H-SC,15.99,90780.0,-7200.0,29068.0,-0.12,-7.35,32.02,22.32,151.0,-0.25,0.71,31.66,XR,ATH,FINANCE
83,UJJIVANSFB,52.77,53.00,36.52,H-SC,20.68,118935.0,-23544.0,24168.0,1.87,-16.52,20.32,0.44,163.0,-0.97,0.92,40.38,OX40N,NTT,BANKS


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.00,20.18,H-SC,13.41,153932.0,27732.0,18441.0,-0.86,21.97,11.98,36.59,156.0,1.50,1.20,37.24,XY24,NTT,HEALTHCARE
51,LALPATHLAB,2838.53,3545.00,-23.68,H-SC,8.05,217138.0,24118.0,23929.0,-0.62,12.50,11.02,24.89,128.0,1.01,1.69,38.76,X40N,NTT,HEALTHCARE
23,COFFEEDAY,59.14,80.00,-51.71,L-SC,21.99,70138.0,-43411.0,83464.0,3.08,-38.23,119.00,35.27,267.0,-0.52,0.55,71.66,XR,NTT,HOTELS
69,SHALBY,261.39,327.00,928.71,M-SC,8.18,139986.0,-41157.0,86623.0,-1.81,-22.72,61.88,25.10,233.0,-0.48,1.09,12.18,XY24,NTT,HEALTHCARE
35,HEROMOTOCO,4311.81,5949.07,0.00,H-MC,6.03,160892.0,9979.0,47318.0,-1.37,6.61,29.41,37.97,90.0,0.21,1.25,33.59,AR,ATH,AUTO


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.49
1,25,41.36
2,50,71.35


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    40.97
LC    36.62
MC    22.42
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.19
X40N     15.36
X40      13.53
XY25     10.86
XR        9.09
OX40N     7.72
AR        7.71
X5K       2.26
MH        1.76
X200      1.40
SR        1.13
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    28.33
H-SC    27.55
H-MC    19.43
M-SC    11.96
M-LC     7.20
M-MC     2.64
L-SC     1.46
L-LC     1.09
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.39,-11.41,47.22
IT,12.10,-19.99,78.27
FINANCE,8.42,-20.52,64.25
BANKS,7.66,-15.76,63.00
PAINTS,5.78,-17.51,40.67
HEALTHCARE,5.19,1.55,23.31
MISC,5.18,-22.15,93.55
ELECTRICAL,4.76,-14.79,64.50
AUTO,4.44,-18.28,69.74


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2886087.0,23
XR,1126195.0,13
AR,1071840.0,9
X40,831385.0,10
XY25,653398.0,8
X40N,629343.0,12
OX40N,384355.0,10
SR,197960.0,2
X5K,148266.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2778632.0,30
H-LC,1510206.0,20
M-SC,1431025.0,15
H-MC,1207840.0,15
M-LC,453783.0,5
M-MC,339071.0,2
L-SC,273073.0,3
L-MC,58039.0,1
L-LC,46367.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      962426.0      7
M-SC       XY24      869737.0      7
H-SC       AR        625608.0      4
H-LC       X40       551806.0      6
H-SC       XR        527144.0      6
H-MC       XY24      475525.0      4
M-MC       XY24      339071.0      2
H-LC       X40N      326000.0      6
           XY25      254776.0      3
           AR        237660.0      2
H-MC       XY25      213685.0      2
H-SC       X40N      212236.0      4
H-MC       X40       203830.0      3
H-SC       SR        197960.0      2
           OX40N     196320.0      6
M-LC       XY24      193092.0      2
L-SC       XR        182723.0      2
M-SC       AR        161254.0      2
M-LC       X5K       134798.0      1
M-SC       XR        125856.0      2
           XY25      122832.0      1
H-MC       XR        122278.0      1
M-LC       XR        110155.0      1
H-MC       X40N       91107.0      2
L-SC       OX40N      90350.0      1
H-LC       X200       80260.0      1
M-SC       X40        75749.0      1
           OX40N      75597.0      2
L-MC       XR         58039.0      1
H-SC       MH         56938.0      1
H-MC       AR         47318.0      1
L-LC       XY25       46367.0      1
H-LC       XY24       46236.0      1
H-MC       MH         32009.0      1
           OX40N      22088.0      1
M-LC       XY25       15738.0      1
H-LC       X5K        13468.0      1

In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
